In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import mwparserfromhell

# Loading Data

And correct some date inconcistencies

### From early version of Emma's code

In [2]:
# Read the file into a list of lines
with open('wiki-RfA.txt', 'r') as file:
    lines = file.readlines()

# Create a list of dictionaries, where each dictionary represents a record
df = []
current_entry = {}

# Iterate through each line, current_entry = one log entry with all columns, df = list of all votee/voter pairs
for line in lines:
    line = line.strip()
    if line:
        key, value = line.split(':', 1)
        current_entry[key] = value
    else:
        df.append(current_entry)
        current_entry = {}

# Append  last record
if current_entry:
    df.append(current_entry)

# Convert into DataFrame and store in csv
df = pd.DataFrame(df)
df.columns = ['Source', 'Target', 'Vote', 'Results', 'Year', 'Date', 'Comment']

df.to_csv('wiki-RfA.csv')

# Set Nan values
# replace field that's entirely space (or empty) with NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# replace inconsistent date
df['Date'] = df['Date'].str.replace('Julu ', 'July ')
df['Date'] = df['Date'].str.replace('Janry ', 'January ')
df['Date'] = df['Date'].str.replace('Mya ', 'May ')
df['Date'] = df['Date'].str.replace('Jan ', 'January ')
df['Date'] = df['Date'].str.replace('Feb ', 'February ')
df['Date'] = df['Date'].str.replace('Mar ', 'March ')
df['Date'] = df['Date'].str.replace('Apr ', 'April ')
df['Date'] = df['Date'].str.replace('Jun ', 'June ')
df['Date'] = df['Date'].str.replace('Jul ', 'July ')
df['Date'] = df['Date'].str.replace('Aug ', 'August ')
df['Date'] = df['Date'].str.replace('Sep ', 'September ')
df['Date'] = df['Date'].str.replace('Oct ', 'October ')
df['Date'] = df['Date'].str.replace('Nov ', 'November ')
df['Date'] = df['Date'].str.replace('Dec ', 'December ')

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%H:%M, %d %B %Y', errors='coerce')

### Adds to do in order to make my (JD) code to work

In [3]:
df.Year = df.Year.astype('Int64')
df.Comment = df.Comment.astype(str)

# Information Extraction from Comments

First create a parsed column with only text infromation from Wikitext (markup language) 

In [4]:
df["Parsed_Comment"] = df.Comment.apply(lambda x: mwparserfromhell.parse(x).strip_code())

# Work in progress

In [ ]:
max_year = df['Year'].max()
min_year = df['Year'].min()
mean_year = df['Year'].mean()

print(f"max_year: {max_year}\n min_year: {min_year}\n mean_year: {mean_year}")

In [ ]:
## Obsolete

df.Comment = df.Comment.astype('str')

def getting_tags_number(comment):
    if "'''" in comment:
        return len(comment.split("'''")) - 2
    else:
        return 0

def getting_tags_list(comment):
    if "'''" in comment[:4]:
        text = comment.split("'''")[1]
        if len(text.split(" ")) > 5:
            return text
        else:   
            return ""
    else:
        return ""

df["CommentTags"] = df.Comment.apply(getting_tags_list)
df["NumberCommentTag"] = df.Comment.apply(getting_tags_number)

for t in df.CommentTags.unique()[250:270]:
    print(t)